**Zum Spielen**

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Als erstes importieren wir unser github Repo. Der nächste Schritt ist der import des Datasets.


In [31]:
!wget https://raw.githubusercontent.com/DeamonAmelie/heart/main/heart.csv


--2021-05-19 16:05:14--  https://raw.githubusercontent.com/DeamonAmelie/heart/main/heart.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11323 (11K) [text/plain]
Saving to: ‘heart.csv’

heart.csv           100%[===================>]  11.06K  --.-KB/s    in 0s      

2021-05-19 16:05:15 (87.4 MB/s) - ‘heart.csv’ saved [11323/11323]



In [34]:
df = pd.read_csv("/content/heart.csv")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


## Beschreibung der Daten

    age - age in years

    sex - sex (1 = male; 0 = female)

    cp - chest pain type (1 = typical angina; 2 = atypical angina; 3 = non-anginal pain; 4 = asymptomatic)

    trestbps - resting blood pressure (in mm Hg on admission to the hospital)

    chol - serum cholestoral in mg/dl

    fbs - fasting blood sugar > 120 mg/dl (1 = true; 0 = false)

    restecg - resting electrocardiographic results (0 = normal; 1 = having ST-T; 2 = hypertrophy)

    thalach - maximum heart rate achieved

    exang - exercise induced angina (1 = yes; 0 = no)

    oldpeak - ST depression induced by exercise relative to rest

    slope - the slope of the peak exercise ST segment (1 = upsloping; 2 = flat; 3 = downsloping)

    ca - number of major vessels (0-3) colored by flourosopy

    thal(thallium stress)- 0 to 3

    Output- 0(less chance of heart attack) and 1(more chance of heart attack)



## EDA 


*   Daten darstellen
*   Daten transformieren ?
* Daten nochmal darstellen
* Korrelation darstellen
* ...

